In [0]:
-- Create new managed volume Landing under dev > bronze

CREATE VOLUME dev.bronze.landing
COMMENT "This is Landing Managed Volume"
;


In [0]:
%python
# Create new folder called input under Landing Volume
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/input")

In [0]:
%python
# Copy retail invoice data from databricks-datasets
dbutils.fs.cp("/databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-01.csv", "/Volumes/dev/bronze/landing/input")

In [0]:
%python
# Copy retail invoice data from databricks-datasets
dbutils.fs.cp("/databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-02.csv", "/Volumes/dev/bronze/landing/input")

In [0]:
-- Create placeholder table dev.bronze.invoice_cp - without any columns
CREATE TABLE IF NOT EXISTS dev.bronze.invoice_cp
;


In [0]:
-- Use copy into to load data into place holder table
COPY INTO dev.bronze.invoice_cp
FROM "/Volumes/dev/bronze/landing/input"
FILEFORMAT = CSV
PATTERN = '*.csv'
FORMAT_OPTIONS(
  'header' = 'true',
  'inferSchema' = 'true',
  'mergeSchema' = 'true'
)
COPY_OPTIONS(
  'mergeSchema' = 'true'
)
;

In [0]:
-- View data from table

select * from dev.bronze.invoice_cp

In [0]:
-- Describe table
describe extended dev.bronze.invoice_cp

In [0]:
-- Create new table with only 3 columns
CREATE OR REPLACE TABLE dev.bronze.invoice_cp_alt(
  InvoiceNo string,
  StockCode string,
  Quantity double,
  _insert_date TIMESTAMP
)
;

In [0]:
-- Load data using COpy INTO 
COPY INTO dev.bronze.invoice_cp_alt
FROM (
  SELECT InvoiceNo, StockCode, cast(Quantity as double), current_timestamp() _insert_date
  FROM
  "/Volumes/dev/bronze/landing/input"
)
FILEFORMAT = CSV
PATTERN = '*.csv'
FORMAT_OPTIONS(
  'header' = 'true',
  'inferSchema' = 'true',
  'mergeSchema' = 'true'
)
;


In [0]:
use catalog `dev`; select * from `bronze`.`invoice_cp_alt` limit 100;